In [ ]:
import pandas as pd
import networkx as nx

# # 加载基因相互作用网络数据
network_data = pd.read_csv('../.././dataSource/39947.protein.links.v12.0.output.txt', sep='\t')
network_data
# 创建图
G = nx.Graph()

# 添加节点和边
for index, row in network_data.iterrows():
    protein1 = row['protein1']
    protein2 = row['protein2']
    combined_score = row['combined_score']
    G.add_edge(protein1, protein2, weight=combined_score)

# 计算介数中心性
betweenness = nx.betweenness_centrality(G, weight='weight')

# 提取原始得分并计算最值
scores = list(betweenness.values())
min_score = min(scores)
max_score = max(scores)

# 处理分母为0的情况（若所有得分相同）
if max_score == min_score:
    scaled_scores = [0] * len(scores)  # 全部设为0（或根据需求调整）
else:
    # 线性归一化到0-100
    scaled_scores = [
        0 + (100 - 0) * (s - min_score) / (max_score - min_score) 
        for s in scores
    ]

# 合并节点和放缩后的得分（保留两位小数）
sorted_items = sorted(betweenness.items(), key=lambda x: x[1], reverse=True)
scaled_results = [
    (node, round(scaled_scores[i], 2)) 
    for i, (node, _) in enumerate(sorted_items)
]

# 保存到CSV
result_df = pd.DataFrame(scaled_results, columns=['gene_id', 'score'])
result_df.to_csv('../.././dataProcessedResult/geneWeight/betweenness_centrality_score.csv', index=False)



,protein1,protein2,combined_score
0,Os09g0104200,Os07g0571100,238
1,Os09g0104200,Os09g0338701,219
2,Os09g0104200,Os01g0737700,506
3,Os09g0104200,Os06g0664800,219
4,Os09g0104200,Os01g0757900,284
...,...,...,...
17730217,Os06g0129700,Os02g0491900,226
17730218,Os06g0129700,Os06g0138400,246
17730219,Os06g0129700,Os06g0634800,366
17730220,Os06g0129700,Os11g0519500,368
